# Install requirements in Google Colab

In [14]:
import pandas as pd

In [5]:
!apt-get update
!apt-get install -y openjdk-17-jdk

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,280 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,311 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [5,665 kB]
Hit:14

In [ ]:
!wget https://github.com/joernio/joern/releases/download/v4.0.324/joern-install.sh
!chmod +x joern-install.sh
!./joern-install.sh --interactive=false


--2025-09-17 14:36:38--  https://github.com/joernio/joern/releases/download/v4.0.324/joern-install.sh
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/175681313/ccc74d91-6eba-4097-9d47-d59859349737?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-09-17T15%3A22%3A10Z&rscd=attachment%3B+filename%3Djoern-install.sh&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-09-17T14%3A21%3A47Z&ske=2025-09-17T15%3A22%3A10Z&sks=b&skv=2018-11-09&sig=8PbbfjUgKEij5dSnhYV34X%2Fw2sVbMDHlU%2BpvnT3H3Uo%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1ODEyMDA5OCwibmJmIjoxNzU4MTE5Nzk4LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi

# run_joern-scan basic implementation
The following implementation of joern-scan is designed to force new cpg generation by default. This behavior can be overridden by passing overwrite=false as the second parameter.<br>
First parameter: directory of files to be scanned. <br>
Second optional parameter: overwrite=Boolean (default=True). To let joern-scan query against a cache copy of the CPG if available, set to false. <br>
Output: Dictionary of result strings parsed from joern-scan ouput. If no vulnerability returns None.

**Note:** Only the first vulnerability found in the scan's output will be returned. Later implementation may add the ability to account for multiple vulnerabilities, but return type may need reconsideration. <br>

Required libraries: os for directory navigation and subprocess to call joern-scan and handle output.

In [1]:
import os
import subprocess

# parameters: directory containing c files to scan and optional boolean overwrite flag which currently defaults to True
def run_joern_scan(directory, overwrite=True):
    results = {}

    # command to execute joern-scan
    call_scan = ['joern-scan']

    # force joern-scan to generate a fresh cpg
    if overwrite:
        call_scan.append('--overwrite')

    # scan every file in supplied directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)

        # won't try to scan directories
        if os.path.isfile(filepath):

            try:
                process = subprocess.run(
                    call_scan + [filepath],
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE,
                    text=True
                )

                # output contains both standard output and any errors
                output = process.stdout + process.stderr
                # clear result from previous scans
                result_line = None
                for line in output.splitlines():
                    # look for the result line and save it
                    if line.startswith("Result:"):
                        result_line = line
                        break

                results[filename] = result_line


                print(f'processed {filename}: {result_line}')

            # handle any errors
            except Exception as e:
                print(f'error processing {filename}: {e}')
                results[filename] = {
                    'result': None,
                    'error': str(e)
                }
    return results

## An example call to the function:

In [7]:
# assign the directory containing code to scan
code_path = "code"

scan_result = run_joern_scan(code_path)


processed boflow1.c: Result: 8.0 : Dangerous function gets() used: boflow1.c:8:get_input


# joern-scan documentation


In [8]:
!joern-scan

Writing logs to: /tmp/joern-scan-log.txt
Creates a code property graph and scans it with queries from installed bundles.
Version: `4.0.421`
Usage: joern-scan [options] [src]

  --help               Prints this usage text
  src                  source code directory to scan
  --overwrite          Overwrite CPG if it already exists
  --store              Store graph changes made by scanner
  --dump               Dump available queries to a file at `/tmp/querydb.json`
  --dump-to <value>    Dump available queries to a specific file
  --list-query-names   Print a list of available query names
  --updatedb           Update query database
  --dbversion <value>  Version of query database `updatedb`-operation installs
  --names <value>      Filter queries used for scanning by name, comma-separated string
  --tags <value>       Filter queries used for scanning by tags, comma-separated string
  --depth <value>      Set call depth for interprocedural analysis
  --language <value>   Source languag

# Example scan of multiple directories and load into dataframe

In [11]:
code_patched_path = "code_patched"

In [29]:
all_results = []

for directory in [code_path, code_patched_path]:
    print(f'scanning {directory}')
    scan_result = run_joern_scan(directory)

    df = pd.DataFrame.from_dict(scan_result, orient='index')
    df['filename'] = df.index
    df['directory'] = directory
    all_results.append(df)

scanning code
processed boflow1.c: Result: 8.0 : Dangerous function gets() used: boflow1.c:8:get_input
scanning code_patched
processed boflow1.c: None


In [30]:
for i in range(len(all_results)):
    print(all_results[i])

                                                           0   filename  \
boflow1.c  Result: 8.0 : Dangerous function gets() used: ...  boflow1.c   

          directory  
boflow1.c      code  
              0   filename     directory
boflow1.c  None  boflow1.c  code_patched


In [31]:
df_all = pd.concat(all_results, ignore_index=True)

In [35]:
df_all

,0,filename,directory
0,Result: 8.0 : Dangerous function gets() used: ...,boflow1.c,code
1,None,boflow1.c,code_patched


In [37]:
df_all = df_all.rename(columns={0: "result:"})

In [38]:
first_col = df_all.columns[0]
print(first_col)


result:


## Saving and loading dataframe to csv file between sessions.

In [18]:
df_all.to_csv('all_results1.csv')

In [19]:
df_all = pd.read_csv('all_results1.csv')

## Function to parse the vulnerability detection output
This converts joern's result string into separate elements each with their own column in the dataframe.

In [22]:
def parse_result_line(result_line):
    """
    Given a result line like:
    "Result: 3.0 : Unchecked read/recv/malloc: fstring4.c:36:main"

    This function returns a tuple with:
    (severity, type, filename, line, caller)

    If the format is unexpected, returns a tuple of Nones.
    """
    if not isinstance(result_line, str) or not result_line.startswith("Result:"):
        return (None, None, None, None, None)

    # Remove the "Result:" prefix and strip whitespace
    trimmed = result_line[len("Result:"):].strip()
    # Split on colons and strip each part
    parts = [part.strip() for part in trimmed.split(':')]

    # Check if we have at least 5 parts
    if len(parts) < 5:
        return (None, None, None, None, None)

    severity = parts[0]
    type_str = parts[1]
    filename = parts[2]
    line_number = parts[3]
    caller = parts[4]

    return (severity, type_str, filename, line_number, caller)


In [39]:


df_all[['severity', 'type', 'filename', 'line', 'caller']] = df_all['result:'].apply(
    lambda x: pd.Series(parse_result_line(x))
)

print(df_all.head())

                                             result:   filename     directory  \
0  Result: 8.0 : Dangerous function gets() used: ...  boflow1.c          code   
1                                               None       None  code_patched   

  severity                            type  line     caller  
0      8.0  Dangerous function gets() used     8  get_input  
1     None                            None  None       None  
